# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 17
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr17.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr17.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 411 411

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000141252,ENSG00000179409,ENSG00000177370,ENSG00000159842,ENSG00000108953,ENSG00000167193,ENSG00000132376,ENSG00000174238,ENSG00000167703,ENSG00000074660
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000141252,1.000000000,-0.002168166,-0.10526912,-0.14260553,-0.11679223,-0.1264286,0.03305758,0.04929982,0.05354040,0.09827782
ENSG00000179409,-0.002168166,1.000000000,0.04153991,0.02523228,-0.07800946,-0.1177687,-0.14877769,-0.09574923,-0.01690717,0.07129504
ENSG00000177370,-0.105269116,0.041539907,1.00000000,-0.01133539,-0.21439141,-0.1725250,0.10247609,-0.13978892,0.00723778,-0.16551010
ENSG00000159842,-0.142605531,0.025232278,-0.01133539,1.00000000,0.09116378,0.1511315,-0.07990859,-0.04184291,0.01421924,-0.05225554
ENSG00000108953,-0.116792232,-0.078009458,-0.21439141,0.09116378,1.00000000,0.1621040,-0.03117966,-0.13869220,-0.07390874,-0.06960261
ENSG00000167193,-0.126428580,-0.117768749,-0.17252496,0.15113150,0.16210402,1.0000000,-0.13947468,-0.04390502,0.01581805,-0.19394277


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 60

[1] -0.02642824 -0.03652306 -0.05615136 -0.05946398 -0.05964009 -0.07255085
 [7] -0.07839822 -0.10174988 -0.11980708 -0.12123773 -0.12496610 -0.14650237
[13] -0.15855579 -0.17128903 -0.20610912 -0.22056899 -0.22567766 -0.24890828
[19] -0.25607502 -0.25888480 -0.26147691 -0.27240369 -0.29444066 -0.31249018
[25] -0.32667842 -0.33790685 -0.37256380 -0.37763915 -0.40278844 -0.42833376
[31] -0.43670109 -0.47066540 -0.47671399 -0.51310147 -0.56048772 -0.59977057
[37] -0.64451516 -0.65301928 -0.68599533 -0.72039667 -0.73077359 -0.82961473
[43] -0.86154053 -0.89796825 -0.95095931 -0.96467252 -1.02496195 -1.04194881
[49] -1.10792360 -1.15248451 -1.18591466 -1.21422690 -1.28089791 -1.29071636
[55] -1.36613435 -1.46650717 -1.56361007 -1.65386645 -1.95994698 -3.95362099

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 411 411

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000141252,ENSG00000179409,ENSG00000177370,ENSG00000159842,ENSG00000108953,ENSG00000167193,ENSG00000132376,ENSG00000174238,ENSG00000167703,ENSG00000074660
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000141252,1.000000000,-0.002168166,-0.10526912,-0.14260553,-0.11679223,-0.1264286,0.03305758,0.04929982,0.05354040,0.09827782
ENSG00000179409,-0.002168166,1.000000000,0.04153991,0.02523228,-0.07800946,-0.1177687,-0.14877769,-0.09574923,-0.01690717,0.07129504
ENSG00000177370,-0.105269116,0.041539907,1.00000000,-0.01133539,-0.21439141,-0.1725250,0.10247609,-0.13978892,0.00723778,-0.16551010
ENSG00000159842,-0.142605531,0.025232278,-0.01133539,1.00000000,0.09116378,0.1511315,-0.07990859,-0.04184291,0.01421924,-0.05225554
ENSG00000108953,-0.116792232,-0.078009458,-0.21439141,0.09116378,1.00000000,0.1621040,-0.03117966,-0.13869220,-0.07390874,-0.06960261
ENSG00000167193,-0.126428580,-0.117768749,-0.17252496,0.15113150,0.16210402,1.0000000,-0.13947468,-0.04390502,0.01581805,-0.19394277


,ENSG00000141252,ENSG00000179409,ENSG00000177370,ENSG00000159842,ENSG00000108953,ENSG00000167193,ENSG00000132376,ENSG00000174238,ENSG00000167703,ENSG00000074660
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000141252,1.0000000000,-0.0002348178,-0.043855656,-0.070361940,-0.06831111,-0.09916495,0.001925025,0.06951742,0.033876487,0.08166539
ENSG00000179409,-0.0002348178,1.0000000000,0.033564392,-0.015451875,-0.03529851,-0.03470117,-0.044539984,-0.04570858,-0.001021914,0.04185624
ENSG00000177370,-0.0438556562,0.0335643924,1.000000000,-0.005327594,-0.07866632,-0.04387253,0.005807192,-0.03517750,-0.013213012,-0.05637138
ENSG00000159842,-0.0703619399,-0.0154518747,-0.005327594,1.000000000,0.06066238,0.09387363,-0.061298733,-0.03078036,0.005949511,-0.05242286
ENSG00000108953,-0.0683111110,-0.0352985141,-0.078666323,0.060662376,1.00000000,0.12835345,-0.033500565,-0.04175059,-0.055257294,-0.03187476
ENSG00000167193,-0.0991649452,-0.0347011691,-0.043872526,0.093873634,0.12835345,1.00000000,-0.052411376,-0.02002554,-0.018207400,-0.09652247


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)